In [ ]:
import tensorflow as tf
import numpy as np

### Experimentation with tf.nn.conv1d

##### input has -> batch_shape + [in_width, in_channels] if data_format is "NWC", or batch_shape + [in_channels, in_width] if data_format is "NCW"

In [ ]:
batch_size = 1
in_width = 1024 # e.g. the sequence length
in_channels = 1 # for 1d, in_channels represents the nr of features 

x_in_shape = (batch_size, in_width, in_channels)

In [ ]:
x_in = tf.constant(tf.random.normal(x_in_shape, stddev=0.1), name="input")

In [ ]:
x_in

In [ ]:
x_in.shape

In [ ]:
x_in.shape[2]

##### filter/kernel tensor of shape [filter_width, in_channels, out_channels]

In [ ]:
kernel_width = 3
out_channels = 12 # nr kernel/filter to apply, equiv to nr of neurons in network

filter_weights_shape = (kernel_width, in_channels, out_channels)
filter_weights_shape

In [ ]:
filter_weights = tf.Variable(tf.random.normal(filter_weights_shape))

In [ ]:
filter_weights

##### Output of applied convolution layer

In [ ]:
conv1d_layer = tf.nn.conv1d(input=x_in, filters=filter_weights, stride=4, padding='SAME')

In [ ]:
conv1d_layer

### Build Custom Conv1D Layer Class

In [ ]:
class Conv1D(tf.Module):
    
    def __init__(self,
               nr_filters: int,
               kernel: int,
               stride: int,
               use_bias: bool,
               name = None
              ):
        
        super(Conv1D, self).__init__(name)
        
        self.nr_filters = nr_filters
        self.kernel = kernel
        self.stride = stride
        self.use_bias = use_bias
        
        self.is_built: bool = False
        
        self.W: tf.Tensor = None 
        self.b: tf.Tensor = None
            
    def __call__(self, x_in):
        
        if not self.is_built:
            in_channels = x_in.shape[-1]
            print(f"in_channels are: {in_channels}")
            filter_weights_shape = (self.kernel, in_channels, self.nr_filters)
            print(f"filter_weights_shape is: {filter_weights_shape}")
            
            self.W = tf.Variable(tf.random.normal(filter_weights_shape, stddev=0.1),
                                trainable=True,
                                dtype = tf.float32,
                                name = "conv1d_filters")
            if self.use_bias:
                self.b = tf.Variable(tf.random.normal([self.nr_filters]))
            
            print(f"filters weights are: {self.W}")
            print(f"bias weights are: {self.b}")
            self.is_built = True
            
        if self.use_bias:
            return tf.add(
                tf.nn.conv1d(
                input=x_in,
                filters=self.W,
                stride=self.stride,
                padding="SAME"
                ),
                self.b,
                name="conv1d_layer_with_bias"
            )
        else:
            return tf.nn.conv1d(
                input=x_in,
                filters=self.W,
                stride=self.stride,
                padding="SAME"
                )

In [ ]:
conv1d = Conv1D(nr_filters=12,
               kernel=2,
               stride=1,
               use_bias=False)

In [ ]:
conv1d(x_in=x_in)